In [110]:
import pandas as pd
import numpy as np

Strompreis=pd.read_csv("Daten entsoe/Gro_handelspreise_202311010000_202502010000_Stunde.csv",na_values='-',header=0 ,sep=';', decimal=",", thousands=".")

In [111]:
Strompreis['Datum']=pd.to_datetime(Strompreis['Datum von'],format='%d.%m.%Y %H:%M')
Strompreis=Strompreis.sort_values('Datum',kind='stable')
Strompreis['Datum']=pd.to_datetime(Strompreis['Datum von'],format='%d.%m.%Y %H:%M')
Strompreis['Datum']=Strompreis['Datum'].dt.tz_localize('Europe/Berlin',ambiguous='infer')
Strompreis['Date']=Strompreis['Datum'].dt.tz_convert('UTC')
Strompreis=Strompreis.set_index('Date')

In [136]:
datei="Daten entsoe/Physikalischer_Stromfluss_202311010000_202502010000_Viertelstunde.csv"
dtype={'Datum von':"string",'Datum bis':"string"
       ,'Nettoexport [MWh] Originalauflösungen':np.float64
       ,'Niederlande (Export) [MWh] Originalauflösungen':np.float64
       ,'Niederlande (Import) [MWh] Originalauflösungen':np.float64
       ,'Schweiz (Export) [MWh] Originalauflösungen':np.float64
       ,'Schweiz (Import) [MWh] Originalauflösungen':np.float64
       ,'Dänemark (Export) [MWh] Originalauflösungen':np.float64
       ,'Dänemark (Import) [MWh] Originalauflösungen':np.float64
       ,'Tschechien (Export) [MWh] Originalauflösungen':np.float64
       ,'Tschechien (Import) [MWh] Originalauflösungen':np.float64
       ,'Luxemburg (Export) [MWh] Originalauflösungen':np.float64
       ,'Luxemburg (Import) [MWh] Originalauflösungen':np.float64
       ,'Schweden (Export) [MWh] Originalauflösungen':np.float64
       ,'Schweden (Import) [MWh] Originalauflösungen':np.float64
       ,'Österreich (Export) [MWh] Originalauflösungen':np.float64
       ,'Österreich (Import) [MWh] Originalauflösungen':np.float64
       ,'Frankreich (Export) [MWh] Originalauflösungen':np.float64
       ,'Frankreich (Import) [MWh] Originalauflösungen':np.float64
       ,'Polen (Export) [MWh] Originalauflösungen':np.float64
       ,'Polen (Import) [MWh] Originalauflösungen':np.float64
       ,'Norwegen (Export) [MWh] Originalauflösungen':np.float64
       ,'Norwegen (Import) [MWh] Originalauflösungen':np.float64
       ,'Belgien (Export) [MWh] Originalauflösungen':np.float64
       ,'Belgien (Import) [MWh] Originalauflösungen':np.float64}


#Stromfluss=pd.read_csv("Daten entsoe/Physikalischer_Stromfluss_202311010000_202502010000_Viertelstunde.csv",sep=';', decimal=",", thousands=".",dtype=dtype,on_bad_lines='skip')
Stromfluss=pd.read_csv(datei,sep=';', decimal=",", thousands=".",header=0,na_values='-',on_bad_lines='skip')

In [150]:
Stromfluss['Datum']=pd.to_datetime(Stromfluss['Datum von'],format='%d.%m.%Y %H:%M')
#Stromfluss=Stromfluss.sort_values('Datum',kind='stable')
Stromfluss['Datum']=pd.to_datetime(Stromfluss['Datum von'],format='%d.%m.%Y %H:%M')
Stromfluss['Datum']=Stromfluss['Datum'].dt.tz_localize('Europe/Berlin',ambiguous='infer')
Stromfluss['Date']=Stromfluss['Datum'].dt.tz_convert('UTC')
Stromfluss=Stromfluss[Stromfluss['Date'].dt.year==2024]
Stromfluss=Stromfluss.set_index('Date').fillna(0)
pm_cat=Stromfluss['Nettoexport [MWh] Originalauflösungen']
Stromfluss['+-']=(np.where(pm_cat<0,'0: Import','1: Export'))
Strompreis=Strompreis.reindex(Stromfluss.index).ffill()
#Stromfluss.rename(columns={'Nettoexport [MWh] Originalauflösungen':'Nettoexport [TWh]'})
Stromfluss['Preis']=Strompreis['Deutschland/Luxemburg [€/MWh] Originalauflösungen']
Stromfluss['Nettoexport [TWh]']=Stromfluss['Nettoexport [MWh] Originalauflösungen']/1000000
Stromfluss['Wert Mio €']=Stromfluss['Preis']*Stromfluss['Nettoexport [MWh] Originalauflösungen']/1000000
#Stromfluss=Stromfluss.set_index('Date')

In [182]:
# funktioniert: 
#Stromfluss.groupby([Stromfluss.index.month,Stromfluss['+-']])[['Nettoexport [TWh]','Wert Mio €']].agg(['sum'])
df=Stromfluss.reset_index()
df['Monat']=df['Date'].dt.month
df['Jahr']=df['Date'].dt.year
Strom=pd.pivot_table(df,index='Monat', columns=['+-'],values=['Wert Mio €','Nettoexport [TWh]'], aggfunc='sum')

In [183]:
#Strom['Durchschnittspreis [€/MWh]','0: Import']=Strom['Wert Mio €']['0: Import']/Strom['Nettoexport [TWh]']['0: Import']
#Strom['Durchschnittspreis [€/MWh]','1: Export']=Strom['Wert Mio €']['1: Export']/Strom['Nettoexport [TWh]']['1: Export']
Strom=Strom.join(Strom[['Wert Mio €']].div(Strom['Nettoexport [TWh]']).rename(columns={'Wert Mio €':'Durchschnittspreis [€/MWh]'}))


In [184]:
Strom

Nettoexport [TWh]            Wert Mio €              \
+-            0: Import 1: Export   0: Import   1: Export   
Monat                                                       
1             -1.134009  2.727416 -109.468334  153.401560   
2             -1.703808  1.916092 -126.664725   72.941809   
3             -2.723841  1.008719 -214.313978   33.724300   
4             -3.103580  0.779169 -272.950747    1.423913   
5             -4.105454  0.678305 -367.218140  -11.308225   
6             -4.055072  0.492438 -378.533584   -3.923504   
7             -4.767093  0.476923 -419.617906   -5.529356   
8             -4.784844  0.505738 -492.243474   -5.031528   
9             -3.162973  0.553136 -336.588938    5.009270   
10            -3.740583  0.348937 -377.071172    4.122459   
11            -2.707387  1.521734 -399.674884  115.197716   
12            -1.958986  2.147534 -405.004888  129.183802   

      Durchschnittspreis [€/MWh]             
+-                     0: Import  1: Export  
Monat                                        
1                      96.532177  56.244294  
2                      74.342124  38.068005  
3                      78.680782  33.432816  
4                      87.947056   1.827475  
5                      89.446414 -16.671293  
6                      93.348178  -7.967512  
7                      88.023860 -11.593819  
8                     102.875564  -9.948883  
9                     106.415369   9.056127  
10                    100.805462  11.814328  
11                    147.623847  75.701611  
12                    206.742131  60.154471

In [132]:
Stromfluss

,Date,Datum von,Datum bis,Nettoexport [MWh] Originalauflösungen,Niederlande (Export) [MWh] Originalauflösungen,Niederlande (Import) [MWh] Originalauflösungen,Schweiz (Export) [MWh] Originalauflösungen,Schweiz (Import) [MWh] Originalauflösungen,Dänemark (Export) [MWh] Originalauflösungen,Dänemark (Import) [MWh] Originalauflösungen,...,Polen (Export) [MWh] Originalauflösungen,Polen (Import) [MWh] Originalauflösungen,Norwegen (Export) [MWh] Originalauflösungen,Norwegen (Import) [MWh] Originalauflösungen,Belgien (Export) [MWh] Originalauflösungen,Belgien (Import) [MWh] Originalauflösungen,Datum,+-,Preis,Wert
0,2023-10-31 23:00:00+00:00,01.11.2023 00:00,01.11.2023 00:15,-2565.0,221.25,0.0,37.5,-262.50,0.0,-192.75,...,0.0,-278.00,0.0,-242.50,0.0,-68.25,2023-11-01 00:00:00+01:00,0: Import,65.37,-167674.050
1,2023-10-31 23:15:00+00:00,01.11.2023 00:15,01.11.2023 00:30,-2464.0,235.00,0.0,52.5,-264.50,0.0,-173.25,...,0.0,-278.00,0.0,-222.75,0.0,-71.25,2023-11-01 00:15:00+01:00,0: Import,65.37,-161071.680
2,2023-10-31 23:30:00+00:00,01.11.2023 00:30,01.11.2023 00:45,-2501.0,233.75,0.0,53.5,-305.75,0.0,-182.25,...,0.0,-278.00,0.0,-223.00,0.0,-71.25,2023-11-01 00:30:00+01:00,0: Import,65.37,-163490.370
3,2023-10-31 23:45:00+00:00,01.11.2023 00:45,01.11.2023 01:00,-2665.5,163.25,0.0,58.5,-389.75,0.0,-171.50,...,0.0,-278.00,0.0,-200.25,0.0,-80.25,2023-11-01 00:45:00+01:00,0: Import,65.37,-174243.735
4,2023-11-01 00:00:00+00:00,01.11.2023 01:00,01.11.2023 01:15,-2298.0,162.25,0.0,51.0,-361.50,0.0,-138.50,...,0.0,-244.75,0.0,-110.25,0.0,-174.25,2023-11-01 01:00:00+01:00,0: Import,65.58,-150702.840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43963,2025-01-31 21:45:00+00:00,31.01.2025 22:45,31.01.2025 23:00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,...,0.0,0.00,0.0,0.00,0.0,0.00,2025-01-31 22:45:00+01:00,1: Export,125.29,0.000
43964,2025-01-31 22:00:00+00:00,31.01.2025 23:00,31.01.2025 23:15,0.0,0.00,0.0,0.0,0.00,0.0,0.00,...,0.0,0.00,0.0,0.00,0.0,0.00,2025-01-31 23:00:00+01:00,1: Export,125.29,0.000
43965,2025-01-31 22:15:00+00:00,31.01.2025 23:15,31.01.2025 23:30,0.0,0.00,0.0,0.0,0.00,0.0,0.00,...,0.0,0.00,0.0,0.00,0.0,0.00,2025-01-31 23:15:00+01:00,1: Export,125.29,0.000
43966,2025-01-31 22:30:00+00:00,31.01.2025 23:30,31.01.2025 23:45,0.0,0.00,0.0,0.0,0.00,0.0,0.00,...,0.0,0.00,0.0,0.00,0.0,0.00,2025-01-31 23:30:00+01:00,1: Export,125.29,0.000


In [128]:
Stromfluss.reset_index()

,Date,Datum von,Datum bis,Nettoexport [MWh] Originalauflösungen,Niederlande (Export) [MWh] Originalauflösungen,Niederlande (Import) [MWh] Originalauflösungen,Schweiz (Export) [MWh] Originalauflösungen,Schweiz (Import) [MWh] Originalauflösungen,Dänemark (Export) [MWh] Originalauflösungen,Dänemark (Import) [MWh] Originalauflösungen,...,Polen (Export) [MWh] Originalauflösungen,Polen (Import) [MWh] Originalauflösungen,Norwegen (Export) [MWh] Originalauflösungen,Norwegen (Import) [MWh] Originalauflösungen,Belgien (Export) [MWh] Originalauflösungen,Belgien (Import) [MWh] Originalauflösungen,Datum,+-,Preis,Wert
0,2023-10-31 23:00:00+00:00,01.11.2023 00:00,01.11.2023 00:15,-2565.0,221.25,0.0,37.5,-262.50,0.0,-192.75,...,0.0,-278.00,0.0,-242.50,0.0,-68.25,2023-11-01 00:00:00+01:00,0: Import,65.37,-167674.050
1,2023-10-31 23:15:00+00:00,01.11.2023 00:15,01.11.2023 00:30,-2464.0,235.00,0.0,52.5,-264.50,0.0,-173.25,...,0.0,-278.00,0.0,-222.75,0.0,-71.25,2023-11-01 00:15:00+01:00,0: Import,65.37,-161071.680
2,2023-10-31 23:30:00+00:00,01.11.2023 00:30,01.11.2023 00:45,-2501.0,233.75,0.0,53.5,-305.75,0.0,-182.25,...,0.0,-278.00,0.0,-223.00,0.0,-71.25,2023-11-01 00:30:00+01:00,0: Import,65.37,-163490.370
3,2023-10-31 23:45:00+00:00,01.11.2023 00:45,01.11.2023 01:00,-2665.5,163.25,0.0,58.5,-389.75,0.0,-171.50,...,0.0,-278.00,0.0,-200.25,0.0,-80.25,2023-11-01 00:45:00+01:00,0: Import,65.37,-174243.735
4,2023-11-01 00:00:00+00:00,01.11.2023 01:00,01.11.2023 01:15,-2298.0,162.25,0.0,51.0,-361.50,0.0,-138.50,...,0.0,-244.75,0.0,-110.25,0.0,-174.25,2023-11-01 01:00:00+01:00,0: Import,65.58,-150702.840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43963,2025-01-31 21:45:00+00:00,31.01.2025 22:45,31.01.2025 23:00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,...,0.0,0.00,0.0,0.00,0.0,0.00,2025-01-31 22:45:00+01:00,1: Export,125.29,0.000
43964,2025-01-31 22:00:00+00:00,31.01.2025 23:00,31.01.2025 23:15,0.0,0.00,0.0,0.0,0.00,0.0,0.00,...,0.0,0.00,0.0,0.00,0.0,0.00,2025-01-31 23:00:00+01:00,1: Export,125.29,0.000
43965,2025-01-31 22:15:00+00:00,31.01.2025 23:15,31.01.2025 23:30,0.0,0.00,0.0,0.0,0.00,0.0,0.00,...,0.0,0.00,0.0,0.00,0.0,0.00,2025-01-31 23:15:00+01:00,1: Export,125.29,0.000
43966,2025-01-31 22:30:00+00:00,31.01.2025 23:30,31.01.2025 23:45,0.0,0.00,0.0,0.0,0.00,0.0,0.00,...,0.0,0.00,0.0,0.00,0.0,0.00,2025-01-31 23:30:00+01:00,1: Export,125.29,0.000
